In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

sys.path.append('/home/peter/Code/deepomics')
from neuralnetwork import NeuralNet, NeuralTrainer
import train as fit 
import visualize, utils

from lasagne import layers, nonlinearities, updates, objectives, init 
from lasagne.layers import get_output, get_output_shape, get_all_params
import theano.tensor as T
import theano
np.random.seed(247) # for reproducibility


Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)


In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
file_path = os.path.join(data_path, filename)
group_name = ['processed_data']
dataset = h5py.File(file_path,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

train = (dtf.transpose([0,3,1,2]), ltf)
valid = (dtf_crossval.transpose([0,3,1,2]), ltf_crossval)


CPU times: user 324 ms, sys: 588 ms, total: 912 ms
Wall time: 911 ms


# original model

In [3]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['conv2'] = layers.Conv2DLayer(net['conv1'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2'], pool_size=(1, 10), stride=(1, 10)) # 25

    net['conv3'] = layers.Conv2DLayer(net['conv2_pool'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['dense1'] = layers.DenseLayer(net['conv3'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.rectify)
    
    net['output'] = layers.DenseLayer(net['dense1'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [4]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.40137 -- accuracy=83.96%  
  valid loss:		0.37519
  valid accuracy:	0.85309+/-0.00006
  valid auc-roc:	0.90830+/-0.00002
  valid auc-pr:		0.89853+/-0.00614
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.37267 -- accuracy=85.35%  
  valid loss:		0.36890
  valid accuracy:	0.85491+/-0.00003
  valid auc-roc:	0.91288+/-0.00002
  valid auc-pr:		0.90612+/-0.00803
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36676 -- accuracy=85.54%  
  valid loss:		0.36356
  valid accuracy:	0.85665+/-0.00001
  valid auc-roc:	0.91432+/-0.00001
  valid auc-pr:		0.90777+/-0.00744
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 4 out of 500 
[==============================] 100.0% -- time=0s -- lo

<neuralnetwork.NeuralTrainer instance at 0x7f26ef154998>

In [5]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.35232
  test  accuracy:	0.86048+/-0.00003
  test  auc-roc:	0.91995+/-0.00000
  test  auc-pr:		0.91446+/-0.00609


0.35231954771865853

# original model w/ batch norm and dropout and relu

In [ ]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1'], nonlinearity=nonlinearities.rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_bells'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


Epoch 1 out of 500 
[================              ] 54.0% -- time=27s -- loss=0.41420 -- accuracy=83.44%  

In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

# modified network (wider + batch norm + dropout + prelu)

In [ ]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1_norm'], alpha=init.Constant(0.25))
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.5)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_bells_whistles'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=1)
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)

In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

# normal residual model

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_2_resid'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1'], nonlinearity=nonlinearities.rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_residual'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=1)
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

# residual model (modified)

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net


    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=50, filter_size=(3, 11), stride=(1, 1),    # 330
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.rectify)
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 32
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=100, filter_size=(1, 6), stride=(1, 1), # 27
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 9), stride=(1, 9)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv2_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1'], nonlinearity=nonlinearities.rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.0003, 
                    "l2": 1e-5
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = '1D_model_resid'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

# 1D architecture

In [44]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(3, 6), stride=(1, 1),    # 330
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_active'], pool_size=(1, 5), stride=(1, 5)) # 65
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(1, 6), stride=(1, 1), # 60
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 5), stride=(1, 5)) # 12
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 4), stride=(1, 1), # 8
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_pool'] = layers.MaxPool2DLayer(net['conv3_active'], pool_size=(1, 3), stride=(1, 3)) # 25
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_pool'], p=0.1)
    
    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=init.Constant(0.1), nonlinearity=None)
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1'], alpha=init.Constant(0.25))
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.0003, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)
savepath = utils.make_directory(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

making directory: ../results/test/1d_version
compiling model


In [45]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.40147 -- accuracy=84.37%  
  valid loss:		0.37892
  valid accuracy:	0.85196+/-0.00007
  valid auc-roc:	0.90785+/-0.00000
  valid auc-pr:		0.89887+/-0.00923
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.37666 -- accuracy=85.37%  
  valid loss:		0.36710
  valid accuracy:	0.85686+/-0.00005
  valid auc-roc:	0.91342+/-0.00000
  valid auc-pr:		0.90614+/-0.00795
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36937 -- accuracy=85.60%  
  valid loss:		0.36464
  valid accuracy:	0.85817+/-0.00000
  valid auc-roc:	0.91527+/-0.00000
  valid auc-pr:		0.90877+/-0.00711
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 4 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36458 -

<neuralnetwork.NeuralTrainer instance at 0x7f4696804b48>

In [46]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.34389
  test  accuracy:	0.86331+/-0.00000
  test  auc-roc:	0.92477+/-0.00000
  test  auc-pr:		0.92022+/-0.00369


0.34388726719256413